In [31]:
import numpy as np
import seaborn as sns
import pandas as pd 
import matplotlib.pyplot as plt


In [32]:
data = pd.read_excel("HR_Employee_Data.xlsx")

In [33]:
data.head()

,Emp_Id,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,IND02438,0.38,0.53,2,157,3,0,1,0,sales,low
1,IND28133,0.80,0.86,5,262,6,0,1,0,sales,medium
2,IND07164,0.11,0.88,7,272,4,0,1,0,sales,medium
3,IND30478,0.72,0.87,5,223,5,0,1,0,sales,low
4,IND24003,0.37,0.52,2,159,3,0,1,0,sales,low


In [34]:
data.drop(['Emp_Id'], axis=1, inplace=True)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   Department             14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [36]:
data.corr().style.background_gradient(sns.light_palette('b', as_cmap=True))

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
satisfaction_level,1.000000,0.105021,-0.142970,-0.020048,-0.100866,0.058697,-0.388375,0.025605
last_evaluation,0.105021,1.000000,0.349333,0.339742,0.131591,-0.007104,0.006567,-0.008684
number_project,-0.142970,0.349333,1.000000,0.417211,0.196786,-0.004741,0.023787,-0.006064
average_montly_hours,-0.020048,0.339742,0.417211,1.000000,0.127755,-0.010143,0.071287,-0.003544
time_spend_company,-0.100866,0.131591,0.196786,0.127755,1.000000,0.002120,0.144822,0.067433
Work_accident,0.058697,-0.007104,-0.004741,-0.010143,0.002120,1.000000,-0.154622,0.039245
left,-0.388375,0.006567,0.023787,0.071287,0.144822,-0.154622,1.000000,-0.061788
promotion_last_5years,0.025605,-0.008684,-0.006064,-0.003544,0.067433,0.039245,-0.061788,1.000000


In [37]:
list(enumerate(data.drop(['left'], 1).columns))

[(0, 'satisfaction_level'),
 (1, 'last_evaluation'),
 (2, 'number_project'),
 (3, 'average_montly_hours'),
 (4, 'time_spend_company'),
 (5, 'Work_accident'),
 (6, 'promotion_last_5years'),
 (7, 'Department'),
 (8, 'salary')]

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

ct = ColumnTransformer([
    ('one-hot', OneHotEncoder(), [7],
    ('ordinal', OrdinalEncoder(), [8]))
], remainder='passthrough')

In [39]:
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [40]:
# !pip install xgboost

In [41]:
pipes = {}
models = {}

models['SVC'] = SVC(probability=True)
models['LR'] = LogisticRegression()
models['DT'] = DecisionTreeClassifier()
models['RF'] = RandomForestClassifier()
models['XGB'] = XGBClassifier(use_label_encoder=False)

for m in models:
    pipes[m] = Pipeline([
        ('columns compose', ct),
        ('standardize', StandardScaler()),
        (m, models[m])
    ])

In [42]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

X, y = data.drop(['left'], axis=1), data['left']

le = LabelEncoder().fit(y)
y = le.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [43]:
for p in pipes:
    pipes[p].fit(X_train, y_train)

ValueError: too many values to unpack (expected 3)